In [1]:
from scipy import stats
import csv 
import sys
import os
sys.path.append("..")
import matplotlib.pyplot as plt
import random
import numpy as np
import math
from astropy.io import fits

from functions import open_csv, separete_csv, both_array_sigma